In [1]:
import pandas as pd
import numpy as np

from route import Route
# from weather_forecast import WeatherForecast
# from api_data import ApiData
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from FUNCTIONS import get_current_location

In [ ]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [ ]:
current_position = {'longitude': 130.868566,
                    'latitude': -12.432466}
delta_spacing = 100000.0 # in meters
number_sites = 3
route = Route()
# route.get_final_data(current_position)
# route.get_final_data(current_position, delta_spacing=delta_spacing)
dataframe = route.get_route_data(current_position, number_sites=number_sites)

In [2]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

Response status from siteinfo: OK.
Current sites' info has been retrieved. 
   name site_id  longitude  latitude altitude
0    0  584987    130.869  -12.4325       18
1    1  584988    133.804  -23.8618      557
2    2  584989    138.609  -34.9352       52


In [ ]:
# api_executer.add_sites(dataframe, print_is_requested=False)

In [ ]:
position = {
    'latitude': -13.0,
    'longitude': 134.0
}
api_requester.get_site_edit(584978, True, name="aa")
api_requester.get_site_edit(584979, True, position=position)
api_requester.get_site_edit(584980, True, name="df", position=position)

In [ ]:
api_requester.get_site_info()

In [ ]:
# api_executer.delete_all_sites()
api_requester.get_site_delete(584978, True)

In [ ]:
api_requester.get_site_info()

In [3]:
forecasts = api_requester.get_solar_forecast()
print(forecasts)

Response status from getforecast: OK.
Solar forecast have been retrieved.
                                     gk      gh_max   gh     tt         ff  \
site_id time                                                                 
584987  2023-08-25 09:00:00   26.415131   37.115045   30  27.80  11.000000   
        2023-08-25 09:10:00   13.474237   16.970465   14  27.60  10.833333   
        2023-08-25 09:20:00    0.000000    5.877645    5  27.40  10.666667   
        2023-08-25 09:30:00    0.000000    0.000000    0  27.20  10.500000   
        2023-08-25 09:40:00    0.000000    0.000000    0  27.00  10.333333   
...                                 ...         ...  ...    ...        ...   
584989  2023-09-03 07:20:00  105.000000  183.686111  105  16.30  16.333333   
        2023-09-03 07:30:00   84.000000  150.551809   84  16.05  16.000000   
        2023-09-03 07:40:00   66.000000  118.673060   66  15.80  15.666667   
        2023-09-03 07:50:00   48.000000   88.791628   48  15.55  15.

In [11]:
# Extract all 'rr' values for all site_id and time combinations
# rr_values = forecasts.loc[:, ('rr', slice(None), slice(None))].droplevel('variable')
rr_values = forecasts.loc[:, ('tt')]
rr_values = rr_values.to_frame()
print(rr_values)
# Assuming your MultiIndex DataFrame is named "forecast_data"
pivot_table = rr_values.pivot(index='time', columns='site_id', values='tt')
print(pivot_table)
pivot_table.to_csv('pivot_table.csv')

print(rr_values)

                                tt
site_id time                      
584987  2023-08-25 09:00:00  27.80
        2023-08-25 09:10:00  27.60
        2023-08-25 09:20:00  27.40
        2023-08-25 09:30:00  27.20
        2023-08-25 09:40:00  27.00
...                            ...
584989  2023-09-03 07:20:00  16.30
        2023-09-03 07:30:00  16.05
        2023-09-03 07:40:00  15.80
        2023-09-03 07:50:00  15.55
        2023-09-03 08:00:00  15.30

[3873 rows x 1 columns]


KeyError: 'time'

In [ ]:
import pandas as pd

# Create a MultiIndex for the DataFrame
index = pd.MultiIndex.from_product([['space1', 'space2'], ['time1', 'time2'], ['variable1', 'variable2']],
                                   names=['space', 'time', 'variable'])

print(index)
# Create an empty DataFrame with the MultiIndex
df_3d = pd.DataFrame(index=index, columns=['data'])

# Fill the DataFrame with values
df_3d.loc[('space1', 'time1', 'variable1')] = 1
df_3d.loc[('space1', 'time1', 'variable2')] = 2
df_3d.loc[('space1', 'time2', 'variable1')] = 3
df_3d.loc[('space1', 'time2', 'variable2')] = 4
df_3d.loc[('space2', 'time1', 'variable1')] = 5
df_3d.loc[('space2', 'time1', 'variable2')] = 6
df_3d.loc[('space2', 'time2', 'variable1')] = 7
df_3d.loc[('space2', 'time2', 'variable2')] = 8

print(df_3d)
